## load required libraries

In [ ]:
!pip install -U langchain-community pypdf bitsandbytes

In [ ]:
!pip install chromadb rouge

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
import time
from nltk.translate.bleu_score import sentence_bleu

from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

In [ ]:

import torch
import pickle
#import umap
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import re
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict

In [ ]:

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer

from langchain_community.vectorstores import Chroma

from langchain.embeddings.base import Embeddings

explore the mims-harvard/Clinical-knowledge-embeddings


In [ ]:
# read in embeddings
embed_mat  = pickle.load(open("full_h_embed_hms.pkl", 'rb')).numpy()

## if the pickle file is not compatible, there's also a csv version (tab-delimited)
#embed_mat = np.loadtxt("full_h_embed_hms.csv")

# mapping file
node_map_df = pd.read_csv("new_node_map_df.csv")

num_embeddings = embed_mat.shape[0]
dim = embed_mat.shape[1]

print("There are %d clinical concept embeddings that are size 1x%d" % (num_embeddings, dim))
ntypes = node_map_df['ntype'].unique()
ind_min = node_map_df['global_graph_index'].min()
ind_max = node_map_df['global_graph_index'].max()

print("The map dataframe contains nodes for the following types of codes: {}".format(ntypes))
print("Each embedding has a corresponding global_graph_index which takes sequential values from {} to {}.".format(ind_min, ind_max))

There are 67124 clinical concept embeddings that are size 1x128
The map dataframe contains nodes for the following types of codes: ['CPT' 'ICD9CM' 'LNC' 'PHECODE' 'RXNORM' 'SNOMEDCT_US' 'ATC4']
Each embedding has a corresponding global_graph_index which takes sequential values from 0 to 67123.


In [ ]:
node_map_df.head()

,ntype,node_name,node_id,global_graph_index
0,CPT,Vaccine,90749,0
1,CPT,Cholera vaccine,90625,1
2,CPT,Endoscopy Procedures on the Rectum,1007622,2
3,CPT,"Sigmoidoscopy, flexible",1007635,3
4,CPT,Intradermal tuberculosis skin test,86580,4


In [ ]:
node_map_df[node_map_df['node_id']=='LA22002-2']

,ntype,node_name,node_id,global_graph_index
16246,LNC,HI,LA22002-2,16246


In [ ]:
len(embed_mat)

67124

In [ ]:
len(embed_mat) #67124
len(embed_mat[0]) #128

128

In [ ]:
# can query by node name or id; make sure to specify the node type if you're
# looking at a specific clinical vocabulary
ind = node_map_df.loc[(node_map_df['node_name'] == 'Asthma') & (node_map_df['ntype'] == 'ICD9CM')]['global_graph_index'].values[0]

#embed_mat[ind]

## Build vector DB

### load clinical knowledge embedding and pdf research paper

In [ ]:

# load clinical embeddings and mapping metadata file
embed_mat = pickle.load(open("full_h_embed_hms.pkl", 'rb')).numpy()
node_map_df = pd.read_csv("new_node_map_df.csv")

In [ ]:

pdf_file="/content/Slamon-DJ-text.pdf"

loader = PyPDFLoader(pdf_file)
documents = loader.load()

In [ ]:

pdf_file="/content/Slamon-DJ-text.pdf"

loader = PyPDFLoader(pdf_file)
documents = loader.load()

for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
    doc.page_content=doc.page_content.replace('\xad', '')

    # Split the documents into smaller text chunks for better retrieval.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [ ]:
#documents

In [ ]:
# Split the documents into smaller text chunks for better retrieval.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [ ]:
docs[0]

Document(metadata={'producer': 'Acrobat Pro 24 Paper Capture Plug-in', 'creator': 'PyPDF', 'creationdate': '2007-01-15T15:29:22+00:00', 'moddate': '2025-03-02T13:15:22-06:00', 'rgid': 'PB:19364043_AS:101511233277955@1401213572478', 'source': '/content/Slamon-DJ-text.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='on January 15, 2007 www.sciencemag.orgDownloaded from  BIOM 255 (Leffert) – Discussion Feb. 1, 2007 Human Breast Cancer: Correlation of  Relapse and Survival with Amplification  of the HER-2/neu Oncogene  DENNIS J. SLAMON, * GARY M. CLARK, STEVEN G. WONG, WENDY J. LEVIN,  AXEL ULLRICH, WILLIAM L. McGUIRE  The HER-2/neu oncog is a member of the erB-like  oncogene family, and IS related to, but distinct from, the  epidermal growth factor receptor. This gene has been  shown to be amplified in human breast cancer cell lines.  In the current study, alterations of the gene in 189  primary human breast cancers were investigated. HER -2/  neu was found to be ampli

In [ ]:
len(docs)

38

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Combind clinical embedding with a text embedding

In [ ]:



class ClinicalEmbeddings:
    """
    Custom embedding class that uses a one-to-one mapping in node_map_df.
    For a given text (assumed to match a 'node_name'), it retrieves the corresponding
    embedding from embed_mat based on the 'global_graph_index'. If multiple matches are found,
    it concatenates them (up to a fixed maximum) and pads if needed.
    """
    def __init__(self, embed_mat: np.ndarray, node_map_df: pd.DataFrame):
        self.embed_mat = embed_mat
        self.node_map_df = node_map_df

    def embed_query(self, text: str, max_matches: int = 6) -> Tuple[List[float], Dict]:
        text_lower = text.lower()
        # use regex to match complete words only
        matches = self.node_map_df[
            self.node_map_df['node_name'].apply(
                lambda x: re.search(r'\b' + re.escape(x.lower()) + r'\b', text_lower) is not None
            )
        ]

        if not matches.empty:
            # sort matches by the length of node_name in descending order
            matches = matches.assign(name_length=matches['node_name'].str.len())
            matches = matches.sort_values(by='name_length', ascending=False)
     #       print("Matched rows:", matches[['node_name', 'node_id', 'global_graph_index']])

            embeddings = []
            matched_names = []
            node_ids = []
            global_graph_indices = []
            # iterate over the sorted matches
            for _, row in matches.iterrows():
                index = int(row['global_graph_index'])
                embeddings.append(np.array(self.embed_mat[index].tolist()))
                matched_names.append(row['node_name'].lower())
                node_ids.append(row['node_id'])
                global_graph_indices.append(row['global_graph_index'])

            # enforce a fixed number of matches by truncating or padding
            if len(embeddings) < max_matches:
                pad_length = max_matches - len(embeddings)
                zero_embedding = np.zeros_like(embeddings[0])
                embeddings.extend([zero_embedding] * pad_length)
                matched_names.extend(["none"] * pad_length)
                node_ids.extend(["none"] * pad_length)
                global_graph_indices.extend(["none"] * pad_length)
            elif len(embeddings) > max_matches:
                embeddings = embeddings[:max_matches]
                matched_names = matched_names[:max_matches]
                node_ids = node_ids[:max_matches]
                global_graph_indices = global_graph_indices[:max_matches]

            # concatenate embeddings into a single fixed-size vector.
            combined_embedding = np.concatenate(embeddings).tolist()
            metadata = {
                "matched_concepts": ", ".join(matched_names),
               "node_ids": ", ".join(map(str, node_ids)),
               "global_graph_indices": ", ".join(map(str, global_graph_indices))
            }
            return combined_embedding, metadata
        else:
            # if no matches: return default embedding and None for node_id and global_graph_index
            default_embedding = self.embed_mat[0].tolist()
            default_metadata = {"matched_concepts": "default", "node_ids": None, "global_graph_indices": None}
            return default_embedding, default_metadata

    def embed_documents(self, texts: List[str]) -> Tuple[List[List[float]], List[Dict]]:
        embeddings = []
        metadatas = []
        for text in texts:
            emb, meta = self.embed_query(text)
            embeddings.append(emb)
            metadatas.append(meta)
        return embeddings, metadatas




In [ ]:


class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, text: str) -> list[float]:
        return self.model.encode(text).tolist()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return [self.embed_query(text) for text in texts]


from typing import List, Tuple, Dict
import numpy as np
from langchain.embeddings.base import Embeddings

class CombinedEmbeddings(Embeddings):
    def __init__(self, clinical_emb: ClinicalEmbeddings, regular_emb: SentenceTransformerEmbeddings):
        self.clinical_emb = clinical_emb
        self.regular_emb = regular_emb

    def combine(self, clinical_embedding: List[float], regular_embedding: List[float]) -> List[float]:
        """Concatenates the clinical embedding vector with the regular embedding vector."""
        return clinical_embedding + regular_embedding

    def embed_query(self, text: str) -> Tuple[List[float], Dict]:
        # Get the clinical embedding and its metadata
        clinical_embedding, clinical_metadata = self.clinical_emb.embed_query(text)
        # Get the regular embedding
        regular_embedding = self.regular_emb.embed_query(text)
        # Concatenate the embeddings
        combined_embedding = self.combine(clinical_embedding, regular_embedding)

        # Build metadata including matched concepts, node_id, and global_graph_index
        metadata = {
            "matched_concepts": clinical_metadata.get("matched_concepts", "default"),
            "node_ids": clinical_metadata.get("node_ids", None),
            "global_graph_indices": clinical_metadata.get("global_graph_indices", None)
        }
        return combined_embedding, metadata

    def embed_documents(self, texts: List[str]) -> Tuple[List[List[float]], List[Dict]]:
        embeddings = []
        metadatas = []
        for text in texts:
            emb, meta = self.embed_query(text)
            embeddings.append(emb)
            metadatas.append(meta)
        return embeddings, metadatas



Combine Function: combine a single clinical embedding (a list of floats) and a regular embedding (another list of floats) and concatenates them.

Dimension: every query returns one clinical vector (e.g. 128*6 dimensions, which is the fixed size after concatenating/padding) plus the regular embedding (e.g. 384 dimensions), yielding a combined vector of 1152 dimensions.

In [ ]:

# instantiate the regular embeddings using SentenceTransformer
regular_emb = SentenceTransformerEmbeddings("all-MiniLM-L6-v2")
# instantiate the clinical embeddings
clinical_emb = ClinicalEmbeddings(embed_mat, node_map_df)
#  the combined embedding model
combined_emb = CombinedEmbeddings(clinical_emb, regular_emb)  # Now an instance of CombinedEmbeddings


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(node_map_df.head())
print(node_map_df.columns)

  ntype                           node_name  node_id  global_graph_index
0   CPT                             Vaccine    90749                   0
1   CPT                     Cholera vaccine    90625                   1
2   CPT  Endoscopy Procedures on the Rectum  1007622                   2
3   CPT             Sigmoidoscopy, flexible  1007635                   3
4   CPT  Intradermal tuberculosis skin test    86580                   4
Index(['ntype', 'node_name', 'node_id', 'global_graph_index'], dtype='object')


In [ ]:
 #get embeddings and metadatas for the pdf research paper
embeddings, metadatas = combined_emb.embed_documents([d.page_content for d in docs])

In [ ]:
metadatas[0]

{'matched_concepts': 'epidermal growth factor receptor, epidermal growth factor, epidermal growth factor, breast cancer, breast cancer, breast cancer',
 'node_ids': 'LP18396-9, LP18547-7, 1427158, LA14283-8, 174.0, 254837009',
 'global_graph_indices': '22057, 22056, 26311, 17249, 24336, 33453'}

save embeddings

In [ ]:
import json

# Assuming metadatas is a list of metadata dictionaries
with open("metadata2.json", "w", encoding="utf-8") as f:   #metadata1 from 5k chunck size and metadata2 is from 1k chunck size
    json.dump(metadatas, f, indent=2)

In [ ]:
#load saved embeddings
import json
# Open and read the metadata file
with open("metadata2.json", "r", encoding="utf-8") as f:
    metadatas = json.load(f)

print(metadatas)

[{'matched_concepts': 'epidermal growth factor receptor, epidermal growth factor, epidermal growth factor, breast cancer, breast cancer, breast cancer', 'node_ids': 'LP18396-9, LP18547-7, 1427158, LA14283-8, 174.0, 254837009', 'global_graph_indices': '22057, 22056, 26311, 17249, 24336, 33453'}, {'matched_concepts': 'breast cancer, breast cancer, breast cancer, pathogenesis, adjustments, lymph node', 'node_ids': 'LA14283-8, 254837009, 174.0, 263547006, 257738001, LP199987-1', 'global_graph_indices': '17249, 33453, 24336, 44451, 39045, 22778'}, {'matched_concepts': 'proliferation, growth factor, localization, fibroblasts, associated, additional', 'node_ids': '30217000, 81286007, LP200047-1, LP7234-0, LP28580-6, LP263791-8', 'global_graph_indices': '35600, 35679, 22753, 22185, 20620, 21089'}, {'matched_concepts': 'platelet-derived growth factor, protein kinase, as a result of, growth factor, identified, macrophage', 'node_ids': '10987005, LP15830-0, 19531003, 81286007, LP29257-0, 58986001

In [ ]:
class CombinedEmbeddingWrapper(Embeddings):
    def __init__(self, combined_emb):
        self.combined_emb = combined_emb

    def embed_query(self, text: str) -> List[float]:
        embedding, _ = self.combined_emb.embed_query(text)
        return embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # call embed_query for each text and return just the embeddings.
        return [self.embed_query(text) for text in texts]

combined_embedding_wrapper = CombinedEmbeddingWrapper(combined_emb)

### Build Chroma Vector DB

In [ ]:
db_save_path3='/content/drive/MyDrive/vector_db_humana3'


Build the DB the first time

In [ ]:
# import shutil


# # Recreate the Chroma collection
# db3 = Chroma.from_texts(
#     texts=[d.page_content for d in docs],
#     embedding=combined_embedding_wrapper,
#     metadatas=[
#         {
#             "matched_concepts": m.get("matched_concepts", ""),
#             "node_ids": m.get("node_ids", ""),
#             "global_graph_indices": m.get("global_graph_indices", "")
#         }
#         for m in metadatas
#     ],
#     persist_directory=db_save_path3,
#     collection_name="rag-humana3",
#  #   client=client
# )


extract the saved vector DB

In [ ]:
db3 = Chroma(
    collection_name="rag-humana3",
    embedding_function=combined_embedding_wrapper,
    persist_directory=db_save_path3,
)

# Check the number of documents loaded
docs_loaded = db3._collection.count()  # or a similar method if provided
print("Documents in collection:", docs_loaded)


<ipython-input-17-96d64e0b451f>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db3 = Chroma(


Documents in collection: 38


In [ ]:
print(db3._collection.count())

38


## Chatbot with llama 3 (8b)

In [ ]:
from langchain.chains import RetrievalQA,StuffDocumentsChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, TextStreamer, pipeline, BitsAndBytesConfig, AutoModelForCausalLM
import torch

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
HUGGING_FACE_TOKEN='hf_xxxx'

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(model_name, token=HUGGING_FACE_TOKEN)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=HUGGING_FACE_TOKEN,
    quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Create the pipeline for text-generation


# custom prompt template
custom_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a clinical expert. Use the following document excerpts to answer the question.
Answer in 1 or 2 concise sentences. If you don't know, simply say "I don't know." Do not repeat yourself.
<|eot_id|><|start_header_id|>user<|end_header_id|>

{context}

Question: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(template=custom_template, input_variables=["context", "question"])



hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,      #  input length
    max_new_tokens=128,  # number of tokens to generate
 #   truncation=True,      # the input is truncated if too long
temperature=0.1
    )

llm = HuggingFacePipeline(pipeline=hf_pipeline)

retriever = db3.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}  # Set the custom prompt here
)
# Example query
query = "How does HER-2/neu amplification affect survival in breast cancer?"
answer = qa_chain.run(query)



Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=128) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
print(answer)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a clinical expert. Use the following document excerpts to answer the question.
Answer in 1 or 2 concise sentences. If you don't know, simply say "I don't know." Do not repeat yourself.
<|eot_id|><|start_header_id|>user<|end_header_id|>
fes, in  breast cancer did not show amplification of these genes (33).  Alterations of non-tyrosine kinase-related proto-oncogenes in these  Table 3. Association between HER-2/neu amplification and disease parame- ters in combined surveys (189 patients). .  Factor Single 2 to S S to 20 >20 Total pt copy copies copies copies  H"""°""" receptor stf#US  ER+ 91 23 14 2 130 0.05  ER- 45 3 6 s 59  PgR+ 73 20 10 3 106 0.06  PgR- 63 6 10 4 83  Tumqr nu (antimetm)  :s2 31 9 4 0 44 0.19  2-5 62 13 7 2 84  >S 23 4 6 3 36  Unknown 20 0 3 2 25  Aae Rt tliRgnosis (yetm)  :sSO 37 13 8 2 60 0.11  >SO 88 13 10 s 116  Unknown 11 0 2 0 13  Number of positive lymph nodes  0 30 0 3 1 34 0.002  1-3 51 7 6 1

### Evaluation

In [ ]:
import time
import random
import json
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge

nltk.download('punkt')

# onitialize ROUGE
rouge = Rouge()

def compute_metrics(response: str, ground_truth: str) -> tuple:
    # tokenize both texts
    reference_tokens = nltk.word_tokenize(ground_truth.lower())
    hypothesis_tokens = nltk.word_tokenize(response.lower())

    # compute BLEU score with smoothing
    smoothing = SmoothingFunction().method1
    bleu = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smoothing)

    # compute ROUGE-L F1 score
    rouge_scores = rouge.get_scores(response, ground_truth)
    rouge_l_f1 = rouge_scores[0]['rouge-l']['f']

    # compute token-level F1 score based on unique token overlap
    ref_set = set(reference_tokens)
    hyp_set = set(hypothesis_tokens)
    common = ref_set.intersection(hyp_set)
    if not hyp_set or not ref_set:
        precision = recall = token_f1 = 0
    else:
        precision = len(common) / len(hyp_set)
        recall = len(common) / len(ref_set)
        token_f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return bleu, rouge_l_f1, token_f1

# evaluate a QA query and compute latency
def evaluate_query(query: str, ground_truth: str) -> dict:
    start_time = time.time()
    response = qa_chain.run(query)
    latency = time.time() - start_time

    bleu, rouge_l_f1, token_f1 = compute_metrics(response, ground_truth)
    return {
        "query": query,
        "response": response,
        "latency": latency,
        "bleu": bleu,
        "rouge_l_f1": rouge_l_f1,
        "token_f1": token_f1
    }

#  queries and their corresponding ground truth answers.
test_evaluations = [

    {
        "query": "How does HER-2/neu amplification affect survival in breast cancer?",
        "ground_truth": (
            "HER-2/neu amplification correlates with shorter disease-free survival and overall survival, "
            "indicating a worse prognosis for breast cancer patients."
        )
    },
    {
        "query": "What role do positive lymph nodes play in the prognosis of breast cancer?",
        "ground_truth": (
            "The number of positive lymph nodes is a key prognostic factor in breast cancer, affecting both relapse and overall survival."
        )
    },
    {
        "query": "How is gene amplification measured in the study?",
        "ground_truth": (
            "Gene amplification was measured using Southern blot analysis with a 32P-labeled HER-2 probe, "
            "followed by dilution analysis and densitometry."
        )
    },
    {
        "query": "What additional prognostic factors were considered in the analysis?",
        "ground_truth": (
            "Other prognostic factors included tumor size, hormonal receptor status, and patient age, "
            "but HER-2/neu amplification remained a strong independent predictor."
        )
    }
]


evaluation_results = []
for item in test_evaluations:
    result = evaluate_query(item["query"], item["ground_truth"])
    evaluation_results.append(result)

#  evaluation results
for result in evaluation_results:
    print(f"Query: {result['query']}")
    print(f"Response: {result['response']}")
    print(f"Latency: {result['latency']:.3f} seconds")
    print(f"BLEU Score: {result['bleu']:.3f}")
    print(f"ROUGE-L F1 Score: {result['rouge_l_f1']:.3f}")
    print(f"Token-level F1 Score: {result['token_f1']:.3f}")
    print("-" * 50)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=128) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=128) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=128) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `e

Query: How does HER-2/neu amplification affect survival in breast cancer?
Response: 
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a clinical expert. Use the following document excerpts to answer the question.
Answer in 1 or 2 concise sentences. If you don't know, simply say "I don't know." Do not repeat yourself.
<|eot_id|><|start_header_id|>user<|end_header_id|>
fes, in  breast cancer did not show amplification of these genes (33).  Alterations of non-tyrosine kinase-related proto-oncogenes in these  Table 3. Association between HER-2/neu amplification and disease parame- ters in combined surveys (189 patients). .  Factor Single 2 to S S to 20 >20 Total pt copy copies copies copies  H"""°""" receptor stf#US  ER+ 91 23 14 2 130 0.05  ER- 45 3 6 s 59  PgR+ 73 20 10 3 106 0.06  PgR- 63 6 10 4 83  Tumqr nu (antimetm)  :s2 31 9 4 0 44 0.19  2-5 62 13 7 2 84  >S 23 4 6 3 36  Unknown 20 0 3 2 25  Aae Rt tliRgnosis (yetm)  :sSO 37 13 8 2 60 0.11  >SO 88 13 10 s 116  Un

(Additional) use the saved metadata from clinical embedding

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA



# create the pipeline for llama3

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
  #  max_length=512,      #  input length
    max_new_tokens=128,  # number of tokens to generate
#    truncation=True,      # the input is truncated if too long
temperature=0.1
    )

llm = HuggingFacePipeline(pipeline=hf_pipeline)


# helper function that takes a document and a metadata dictionary
def format_doc(doc, meta) -> str:
    meta_str = (
        f"Matched Concepts: {meta.get('matched_concepts', 'N/A')}, "
        f"Node IDs: {meta.get('node_ids', 'N/A')}, "
        f"Global Graph Indices: {meta.get('global_graph_indices', 'N/A')}"
    )
    return f"{meta_str}\nContent: {doc.page_content}"

# custom prompt template that includes metadata in the context.

custom_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a clinical expert. Use the following document excerpts (with associated metadata)  to answer the question.
Answer in 1 or 2 concise sentences. If you don't know, simply say "I don't know." Do not repeat yourself.
<|eot_id|><|start_header_id|>user<|end_header_id|>

{context}

Question: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(template=custom_template, input_variables=["context", "question"])

# create the QA chain using the custom prompt.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db3.as_retriever(search_kwargs={"k": 1})
)

# example query
query = "How does HER-2/neu amplification affect survival in breast cancer?"

# retrieve relevant documents using the retriever
retrieved_docs = db3.as_retriever(search_kwargs={"k": 1}).get_relevant_documents(query)

# format the retrieved documents using the global metadatas list.
# it assumes the order of retrieved_docs corresponds to the order of items in metadatas.
formatted_docs = []
for i, doc in enumerate(retrieved_docs):
    meta = metadatas[i] if i < len(metadatas) else {}
    formatted_docs.append(format_doc(doc, meta))

# join the formatted documents into one context string.
context = "\n\n".join(formatted_docs)

#  the final prompt using our custom prompt template.
final_prompt = prompt.format(context=context, question=query)
print("Final Prompt:\n", final_prompt)





Device set to use cuda:0


Final Prompt:
 
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a clinical expert. Use the following document excerpts (with associated metadata)  to answer the question.
Answer in 1 or 2 concise sentences. If you don't know, simply say "I don't know." Do not repeat yourself.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Matched Concepts: epidermal growth factor receptor, epidermal growth factor, epidermal growth factor, breast cancer, breast cancer, breast cancer, Node IDs: LP18396-9, LP18547-7, 1427158, LA14283-8, 174.0, 254837009, Global Graph Indices: 22057, 22056, 26311, 17249, 24336, 33453
Content: fes, in  breast cancer did not show amplification of these genes (33).  Alterations of non-tyrosine kinase-related proto-oncogenes in these  Table 3. Association between HER-2/neu amplification and disease parame- ters in combined surveys (189 patients). .  Factor Single 2 to S S to 20 >20 Total pt copy copies copies copies  H"""°""" receptor stf#US  ER+ 91 2

In [ ]:
#  LLM to generate an answer based on the final prompt.
answer = llm(final_prompt)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print("Answer:", answer)


Answer: 
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a clinical expert. Use the following document excerpts (with associated metadata)  to answer the question.
Answer in 1 or 2 concise sentences. If you don't know, simply say "I don't know." Do not repeat yourself.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Matched Concepts: epidermal growth factor receptor, epidermal growth factor, epidermal growth factor, breast cancer, breast cancer, breast cancer, Node IDs: LP18396-9, LP18547-7, 1427158, LA14283-8, 174.0, 254837009, Global Graph Indices: 22057, 22056, 26311, 17249, 24336, 33453
Content: fes, in  breast cancer did not show amplification of these genes (33).  Alterations of non-tyrosine kinase-related proto-oncogenes in these  Table 3. Association between HER-2/neu amplification and disease parame- ters in combined surveys (189 patients). .  Factor Single 2 to S S to 20 >20 Total pt copy copies copies copies  H"""°""" receptor stf#US  ER+ 91 23 14 2 

Evaluation

In [ ]:
import time
import json
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge


# initialize ROUGE
rouge = Rouge()

def compute_metrics(response: str, ground_truth: str) -> tuple:
    # tokenize both texts.
    reference_tokens = nltk.word_tokenize(ground_truth.lower())
    hypothesis_tokens = nltk.word_tokenize(response.lower())

    # compute BLEU score with smoothing.
    smoothing = SmoothingFunction().method1
    bleu = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smoothing)

    # compute ROUGE-L F1 score.
    rouge_scores = rouge.get_scores(response, ground_truth)
    rouge_l_f1 = rouge_scores[0]['rouge-l']['f']

    # compute token-level F1 based on unique token overlap.
    ref_set = set(reference_tokens)
    hyp_set = set(hypothesis_tokens)
    common = ref_set.intersection(hyp_set)
    if not hyp_set or not ref_set:
        precision = recall = token_f1 = 0
    else:
        precision = len(common) / len(hyp_set)
        recall = len(common) / len(ref_set)
        token_f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return bleu, rouge_l_f1, token_f1

def evaluate_query_with_metadata(query: str, ground_truth: str) -> dict:
    # dtart timer to measure latency.
    start_time = time.time()
    retrieved_docs = db3.as_retriever(search_kwargs={"k": 1}).get_relevant_documents(query)

# format the retrieved documents using the global metadatas list.
# it assumes the order of retrieved_docs corresponds to the order of items in metadatas.
    formatted_docs = []
    for i, doc in enumerate(retrieved_docs):
      meta = metadatas[i] if i < len(metadatas) else {}
      formatted_docs.append(format_doc(doc, meta))

# join the formatted documents into one context string.
    context = "\n\n".join(formatted_docs)

# build the final prompt using our custom prompt template.
    final_prompt = prompt.format(context=context, question=query)

    response =llm(final_prompt)
    latency = time.time() - start_time

    # Compute evaluation metrics.
    bleu, rouge_l_f1, token_f1 = compute_metrics(response, ground_truth)

    # retrieve documents in context that includes metadata

    retrieved_metadata =  context

    return {
        "query": query,
        "response": response,
        "latency": latency,
        "bleu": bleu,
        "rouge_l_f1": rouge_l_f1,
        "token_f1": token_f1,
        "retrieved_metadata": retrieved_metadata
    }

# sets of test evaluations.
test_evaluations = [

    {
        "query": "How does HER-2/neu amplification affect survival in breast cancer?",
        "ground_truth": (
            "HER-2/neu amplification correlates with shorter disease-free survival and overall survival, "
            "indicating a worse prognosis for breast cancer patients."
        )
    },
    {
        "query": "What role do positive lymph nodes play in the prognosis of breast cancer?",
        "ground_truth": (
            "The number of positive lymph nodes is a key prognostic factor in breast cancer, affecting both relapse and overall survival."
        )
    },
    {
        "query": "How is gene amplification measured in the study?",
        "ground_truth": (
            "Gene amplification was measured using Southern blot analysis with a 32P-labeled HER-2 probe, "
            "followed by dilution analysis and densitometry."
        )
    },
    {
        "query": "What additional prognostic factors were considered in the analysis?",
        "ground_truth": (
            "Other prognostic factors included tumor size, hormonal receptor status, and patient age, "
            "but HER-2/neu amplification remained a strong independent predictor."
        )
    }
]

# evaluate each query and store results.
evaluation_results = []
for item in test_evaluations:
    result = evaluate_query_with_metadata(item["query"], item["ground_truth"])
    evaluation_results.append(result)

# print evaluation results.
for result in evaluation_results:
    print(f"Query: {result['query']}")
    print(f"Response: {result['response']}")
    print(f"Latency: {result['latency']:.3f} seconds")
    print(f"BLEU Score: {result['bleu']:.3f}")
    print(f"ROUGE-L F1 Score: {result['rouge_l_f1']:.3f}")
    print(f"Token-level F1 Score: {result['token_f1']:.3f}")
    print("Retrieved Metadata:", result['retrieved_metadata'])
    print("-" * 50)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query: How does HER-2/neu amplification affect survival in breast cancer?
Response: 
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a clinical expert. Use the following document excerpts (with associated metadata)  to answer the question.
Answer in 1 or 2 concise sentences. If you don't know, simply say "I don't know." Do not repeat yourself.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Matched Concepts: epidermal growth factor receptor, epidermal growth factor, epidermal growth factor, breast cancer, breast cancer, breast cancer, Node IDs: LP18396-9, LP18547-7, 1427158, LA14283-8, 174.0, 254837009, Global Graph Indices: 22057, 22056, 26311, 17249, 24336, 33453
Content: fes, in  breast cancer did not show amplification of these genes (33).  Alterations of non-tyrosine kinase-related proto-oncogenes in these  Table 3. Association between HER-2/neu amplification and disease parame- ters in combined surveys (189 patients). .  Factor Single 2 to S S to 20 >20 T

### template for the Chatbot

In [ ]:
import torch
import re
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

def create_qa_chatbot(model_name: str, hf_token: str, db3, search_k: int = 1):
    """
    Sets up a QA chatbot using the specified model and Chroma vector store.

    Parameters:
      model_name (str): Model identifier (e.g., "meta-llama/Llama-3.1-8B-Instruct").
      hf_token (str): Your Hugging Face access token.
      db3: A pre-loaded Chroma vector store.
      search_k (int): Number of documents to retrieve.

    Returns:
      A function that runs an interactive Q&A loop.
    """

    # configure quantization for 4-bit inference.
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # load tokenizer and model with your token.
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        token=hf_token,
        quantization_config=bnb_config
    )

    #   custom prompt template in Llama-3 style.
    custom_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a clinical expert. Use the following document excerpts to answer the question.
Answer in 1 or 2 concise sentences. If you don't know, simply say "I don't know." Do not repeat yourself.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{context}

Question: {question}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
    prompt = PromptTemplate(template=custom_template, input_variables=["context", "question"])

    # create the Hugging Face pipeline.
    hf_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=128,  # Number of tokens to generate
        temperature=0.1
    )

    # wrap the pipeline for LangChain.
    llm = HuggingFacePipeline(pipeline=hf_pipeline)

    # set up the retriever from saved Chroma vector store.
    retriever = db3.as_retriever(search_kwargs={"k": search_k})

    # create the QA chain using the "stuff" chain type and our custom prompt.
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}
    )

    # helper function to extract the answer text after the assistant header token.
    def extract_answer(raw_text: str) -> str:
        parts = re.split(r"<\|start_header_id\|>assistant<\|end_header_id\|>", raw_text)
        if len(parts) > 1:
            return parts[1].strip()
        return raw_text.strip()

    # interactive Q&A loop function.
    def run_chatbot():
        while True:
            query = input("Enter your question (or type 'exit' to quit): ")
            if query.lower() == "exit":
                print("Exiting Q&A.")
                break
            raw_answer = qa_chain.run(query)
            answer = extract_answer(raw_answer)
            print("Answer:", answer)

    return run_chatbot




In [ ]:
# usage:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
HUGGING_FACE_TOKEN = 'hf_xxxx'
# db3 is the loaded Chroma vector store
run_chatbot = create_qa_chatbot(model_name, HUGGING_FACE_TOKEN, db3, search_k=1)
run_chatbot()

## Chatboat with google T5

In [ ]:
from transformers import AutoTokenizer, LEDForConditionalGeneration, pipeline
from langchain.llms import HuggingFacePipeline



model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



custom_template = """
You are a clinical expert. Use the following document excerpts to answer the question.
Answer in 1 or 2 sentences. If you don't know, just say I don't know. Do not make up answers.

{context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=custom_template, input_variables=["context", "question"])

# create the pipeline for text2text-generation
hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,      #  input length
  #  max_new_tokens=512,  # number of tokens to generate
   # truncation=True,
temperature=0.1
    )

llm = HuggingFacePipeline(pipeline=hf_pipeline)

retriever = db3.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}  # Set the custom prompt here
)

# Example query
query = "How does HER-2/neu amplification affect survival in breast cancer?"
answer = qa_chain.run(query)



Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
print(answer)

I don't know


### Evaluation

In [ ]:
import time
import random
import json
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge



# initialize ROUGE
rouge = Rouge()

def compute_metrics(response: str, ground_truth: str) -> tuple:
    # tokenize both texts
    reference_tokens = nltk.word_tokenize(ground_truth.lower())
    hypothesis_tokens = nltk.word_tokenize(response.lower())

    # compute BLEU score with smoothing
    smoothing = SmoothingFunction().method1
    bleu = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smoothing)

    # compute ROUGE-L F1 score
    rouge_scores = rouge.get_scores(response, ground_truth)
    rouge_l_f1 = rouge_scores[0]['rouge-l']['f']

    # compute token-level F1 score based on unique token overlap
    ref_set = set(reference_tokens)
    hyp_set = set(hypothesis_tokens)
    common = ref_set.intersection(hyp_set)
    if not hyp_set or not ref_set:
        precision = recall = token_f1 = 0
    else:
        precision = len(common) / len(hyp_set)
        recall = len(common) / len(ref_set)
        token_f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return bleu, rouge_l_f1, token_f1

# evaluate a QA query and compute latency
def evaluate_query(query: str, ground_truth: str) -> dict:
    start_time = time.time()
    response = qa_chain.run(query)
    latency = time.time() - start_time

    bleu, rouge_l_f1, token_f1 = compute_metrics(response, ground_truth)
    return {
        "query": query,
        "response": response,
        "latency": latency,
        "bleu": bleu,
        "rouge_l_f1": rouge_l_f1,
        "token_f1": token_f1
    }

# queries and their corresponding ground truth answers.
test_evaluations = [

    {
        "query": "How does HER-2/neu amplification affect survival in breast cancer?",
        "ground_truth": (
            "HER-2/neu amplification correlates with shorter disease-free survival and overall survival, "
            "indicating a worse prognosis for breast cancer patients."
        )
    },
    {
        "query": "What role do positive lymph nodes play in the prognosis of breast cancer?",
        "ground_truth": (
            "The number of positive lymph nodes is a key prognostic factor in breast cancer, affecting both relapse and overall survival."
        )
    },
    {
        "query": "How is gene amplification measured in the study?",
        "ground_truth": (
            "Gene amplification was measured using Southern blot analysis with a 32P-labeled HER-2 probe, "
            "followed by dilution analysis and densitometry."
        )
    },
    {
        "query": "What additional prognostic factors were considered in the analysis?",
        "ground_truth": (
            "Other prognostic factors included tumor size, hormonal receptor status, and patient age, "
            "but HER-2/neu amplification remained a strong independent predictor."
        )
    }
]


evaluation_results = []
for item in test_evaluations:
    result = evaluate_query(item["query"], item["ground_truth"])
    evaluation_results.append(result)

# evaluation results
for result in evaluation_results:
    print(f"Query: {result['query']}")
    print(f"Response: {result['response']}")
    print(f"Latency: {result['latency']:.3f} seconds")
    print(f"BLEU Score: {result['bleu']:.3f}")
    print(f"ROUGE-L F1 Score: {result['rouge_l_f1']:.3f}")
    print(f"Token-level F1 Score: {result['token_f1']:.3f}")
    print("-" * 50)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or uns

Query: How does HER-2/neu amplification affect survival in breast cancer?
Response: I don't know
Latency: 3.226 seconds
BLEU Score: 0.000
ROUGE-L F1 Score: 0.000
Token-level F1 Score: 0.000
--------------------------------------------------
Query: What role do positive lymph nodes play in the prognosis of breast cancer?
Response: I don't know
Latency: 3.997 seconds
BLEU Score: 0.000
ROUGE-L F1 Score: 0.000
Token-level F1 Score: 0.000
--------------------------------------------------
Query: How is gene amplification measured in the study?
Response: using v erbB as a probe
Latency: 3.970 seconds
BLEU Score: 0.004
ROUGE-L F1 Score: 0.167
Token-level F1 Score: 0.231
--------------------------------------------------
Query: What additional prognostic factors were considered in the analysis?
Response: I don't know
Latency: 3.263 seconds
BLEU Score: 0.000
ROUGE-L F1 Score: 0.000
Token-level F1 Score: 0.000
--------------------------------------------------


## Summary

It has shown that Llama 3 (3.1 8b) outperformed Google T5 on the Q&A case. Overall, the evaluation showed mixed performance (given the 4 testing questions): for the query on survival impact, the chatbot correctly identified that HER-2/neu amplification is associated with poorer prognosis and a higher number of positive lymph nodes, though its BLEU, ROUGE-L, and token-level F1 scores were very low, indicating limited overlap with the ground truth. For the question regarding the role of positive lymph nodes, it produced a more focused answer noting the significance of having more than three nodes, with slightly improved metrics. However, when asked about additional prognostic factors, the model repeatedly defaulted to "I don't know," reflecting a gap in its context understanding or retrieval process. Latency ranged from around 5.7 to 9.7 seconds per query, indicating resaonble response times.

To test more comprehensively, I thonk can build a test set of many diverse query-ground truth pairs and automate the evaluation. For example, can store a list/DF of queries and expected answers, then loop through them to compute evaluation metrics (BLEU, ROUGE, token-level F1, latency, etc.), aggregate the results, and generate summary statistics and visualizations.